# Oligopaints based multiplexed RNA-FISH probe sequence design pipeline 
  
The goal of this pipeline is to take a set of genes and use [OligoPaints](https://oligopaints.hms.harvard.edu/genome-files) for encoding probes, [DNA 20-mers](https://elledge.hms.harvard.edu/?page_id=638) for readout probes/primers, and [5x5 bDNA sequences](https://static-content.springer.com/esm/art%3A10.1038%2Fs41598-019-43943-8/MediaObjects/41598_2019_43943_MOESM2_ESM.xlsx) for signal amplification. Signal amplification is necessary because the number of probes for many of our genes of interest are significantly less than the 96 probes used in the original MERFISH papers. The rules for our pipeline are assembled from all of the various MERFISH publications by Rory Kruithoff.  
  
This pipeline can generate readout strategies using a) multiplex by sequential or b) multiplex by barcoding. For (a) the relative expression levels for target genes needs to be provided. For (b) the Hamming code strategy needs to be defined.
  
Written on Ubuntu 18.04 LTS (both native and using Windows Subsytem). This code requires a working local BLAST install, working local BEDtools install, and some work to get cruzdb running with Python 3.x. Will write up the install in an another markdown once everything is finalized.

Current external dependencies:
- local [BLAST install](https://ftp.ncbi.nlm.nih.gov/blast/executables/blast+/LATEST/)
- local [BEDTOOLS install](https://bedtools.readthedocs.io/en/latest/content/installation.html)
  
Current library dependencies:
- python = 3.6
- cruzdb (requires some effort to install in python 3.6, need to use [pull request 16](https://github.com/brentp/cruzdb/pull/16))
- pandas
- pybedtools
- biopython  
- numpy  
- os  
  
Current external data dependencies:
- hg38 OligoPaints BED files  
- hg38 transcriptome fasta file  
- hg38 ncRNA fasta file  
- Elledge lab 240k list of 25-mer sequences
- Zhuang lab modified hamming codes  
- Wollman lab modified hamming codes
- Moffitt lab standard readout sequences
- Moffitt lab amplified readout sequences 

Douglas Shepherd, PhD  
Quantitative Imaging and Inference Lab (qi2lab)  
Center for Biological Physics and Department of Physics  
Arizona State University  
04.2020

## Define probe set parameters
- readout strategy (sequential,barcode) AND (denovo, Moffitt, amplified)
  - if sequential, need to define number of colors per round AND relative expression of genes
  - if barcode, need to define hamming code set (14-bit,16-bit,18-bit,24-bit)
- genome (hg38)
  - TO DO: add support for rat and mouse)
- genes of interest
  - TO DO: add support for reading in from file or other pipeline

In [1]:
# define readout probe strategy
readout_strategy = ['sequential','amplified']

# if using multiplex by barcoding, define barcode strategy
hamming_bit = '16-bit'

# if using multiplex by sequential, define number of colors per round
number_of_colors = 4

# define target genome
genome = 'hg38'

# define target genes
# use refGene ID is UCSC
# TO DO: create function to parse & load output of gene selection software
gene_ids=['CLDN5','PTPRC','PDGFRB','CDH1']

# if using multiplex by sequential, define relative expression of genes by index from high to low
relative_expression = [1,2,3,4]

## Imports

In [2]:
# cruzdb imports
# # https://github.com/brentp/cruzdb/tree/pull16/cruzdb
from cruzdb import Genome 

# pandas imports
# https://pandas.pydata.org/
import pandas as pd 

# pybedtools imports
# https://daler.github.io/pybedtools/
# relies on a local BEDTOOLS installation
import pybedtools 

# biopython imports
# https://biopython.org/
# relies on a local BLASTx installation
from Bio import SeqIO
from Bio import SearchIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Alphabet import generic_dna
from Bio.SeqUtils import MeltingTemp as mt
from Bio.SeqUtils import GC as gcCheck
from Bio.Blast.Applications import NcbimakeblastdbCommandline
from Bio.Blast.Applications import NcbiblastnCommandline as blastn
from Bio.Blast import NCBIXML

# numpy imports
import numpy as np

# os imports
import os
from datetime import datetime

## Encoding probe design 
1. Define genes using refGene name from UCSC browser
2. Pull all isoforms from UCSC associated with refGene name
3. Parse out exons for each isoform.
4. Using OligoPaints 'balanced' hg38 database to select probes for each isoform.
5. Find unique probes that span all isoforms. Save probes in Pandas structure and a BLAST database

In [3]:
# function to take a dataframe of chromosome, exon locations, and strandness for one gene isoform and return a BED file 
def df_to_bed_isoform(gene_id,df_isoform):

    # loop over each exon and create list of strings with format
    # CHR START STOP NAME STRAND
    bed_record=[]
    i=0
    for index, row in df_isoform.iterrows():
        line=(str(row['chromosome']),str(row['start']),str(row['stop']),
              gene_id+'_exon_'+str(row['exon']),0,str(row['strand']))
        bed_record.append(line)
        i+=1

    # convert list of strings to BED record
    BED_isoform = pybedtools.BedTool(bed_record)

    # return BED record
    return BED_isoform

In [4]:
# function to take an isoform and return a set of probes
def find_probes_from_isoform(gene_id,df_isoform,isoform_number):
    
    # convert dataframe to BED record
    BED_isoform = df_to_bed_isoform(gene_id,df_isoform)
    
    # load OligoPaints BED file corresponding to chromosome for gene
    BED_all_probes =pybedtools.BedTool('oligodb/hg38b/hg38_'+str(df_isoform.chromosome.unique()[0]).split('_')[0]+'b.bed')
    
    # find probes using intersect
    encoding_probes = BED_all_probes.intersect(BED_isoform,f=1)
    
    # turn output into a list of strings
    # check if strand is + or -
    # if +, store probe
    # if -, take reverse complement before storing probe
    encoding_probes_sequence = []
    for interval in encoding_probes:
        if df_isoform.strand.unique()[0]=='+':
            encoding_probes_sequence.append(interval.name)
        else:
            temp_seq = Seq(interval.name, generic_dna)
            encoding_probes_sequence.append(str(temp_seq.reverse_complement()))
        
    # convert list of strings into dataframe
    i=0
    df_isoform_probes = pd.DataFrame(columns=['gene','isoform','probe','sequence'])
    for probe_seq in encoding_probes_sequence:
        df_isoform_probes = df_isoform_probes.append({'gene': gene_id, 'isoform': isoform_number,'probe': i,
                                             'sequence': probe_seq},ignore_index=True)
        i+=1
    
    # return dataframe for probes for this gene
    return df_isoform_probes

In [5]:
# function to retrieve exons from UCSC for a refGene ID
def generate_encoding_probes(database, gene_ids):

    # open connection to UCSC database
    # here, we use the hg38 human genome assembly
    g = Genome(db=genome)

    # create empty dataframe to store all probes
    df_encoding_probes = pd.DataFrame(columns=['gene','probe','sequence'])

    # create empty dataframe
    df_encoding_across_all_isoforms = pd.DataFrame(columns=['gene','isoform','probe','sequence'])

    # loop over all genes
    for gene_id in gene_ids:

        # pull all entries for a given gene from UCSC
        gene_entry_all = g.refGene.filter_by(name2=gene_id).all()

        j=0
        # loop over all gene entries
        for isoform in gene_entry_all:

            # extract exons for this gene entry
            exons=isoform.exons

            # create empty dataframe
            df_exons = pd.DataFrame(columns=['gene','chromosome','exon','start','stop','strand'])

            # place each exon in dataframe 
            i=0
            for exon in exons:
                df_exons = df_exons.append({'gene': gene_id, 'chromosome': isoform.chrom, 
                                           'exon': i, 'start': exon[0], 'stop': exon[1], 'strand': isoform.strand},
                                          ignore_index=True)
                i+=1

            df_encoding_isoform=find_probes_from_isoform(gene_id,df_exons,j)
            j+=1

            df_encoding_across_all_isoforms = df_encoding_across_all_isoforms.append(df_encoding_isoform,ignore_index=True)

    df_encoding_unique = df_encoding_across_all_isoforms.drop_duplicates(['sequence'])
    df_encoding_unique=df_encoding_unique.reset_index()

    # place this into the larger dataframe
    df_encoding_probes=df_encoding_probes.append(df_encoding_unique,ignore_index=True)
    df_encoding_probes=df_encoding_probes.drop(columns=['probe','index'])

    return df_encoding_probes

## Parse non-coding RNA FASTA
1. [Download hg38 ncRNA fasta](ftp://ftp.ensembl.org/pub/release-99/fasta/homo_sapiens/ncrna/Homo_sapiens.GRCh38.ncrna.fa.gz)
2. Check if BLASTDB already created
3. Parse out 'tRNA', 'Mt-tRNA', 'rRNA'
4. Create blast database

In [6]:
def fasta_to_blastDB_ncRNA():
    
    if not(os.path.exists('/home/dps/merfish/blastdb/tRNA/tRNA_parsed.fa')):

        records_to_keep=[]
        with open('/home/dps/merfish/blastdb/tRNA/Homo_sapiens.GRCh38.ncrna.fa', 'r') as handle:
            for record in SeqIO.parse(handle, 'fasta'):
                description=record.description
                if ('tRNA' in description) or ('rRNA' in description):
                    records_to_keep.append(record)

        with open('/home/dps/merfish/blastdb/tRNA/tRNA_parsed.fa','a') as output_handle:
            for record in records_to_keep:
                SeqIO.write(record,output_handle,'fasta')

        cline = NcbimakeblastdbCommandline(dbtype='nucl',input_file='/home/dps/merfish/blastdb/tRNA/tRNA_parsed.fa',
                                           title='tRNA',out='/home/dps/merfish/blastdb/tRNA/db/tRNA')
        stdout, stderr = cline()

## Denovo readout probe design
1. Create set of all potential 20-mer readout probes from [known set](https://doi.org/10.1073/pnas.0812506106) of 240,000 25-mers.
2. Select probes with only 'A', 'T', and 'C'.
3. Select probes without 'CCC', 'AAA', and 'TTT'.
4. Select probes with 40-50% GC content.
5. BLAST 20-mers against transcriptome for species of interest. Select those with less than 11 contiguous base homology.
6. BLAST 20-mers aganist tRNA, rRNA for species of interest and mitochondria. Select those with less than 11 contiguous base homology.
7. BLAST 20-mers against other selected 20-mers. Select those with less than 11 contiguous base homology.

In [7]:
def generate_denovo_readout_probes():
    
    # Make sure non-coding RNA database exists
    fasta_to_blastDB_ncRNA()
    
    big_list_25mers_all = list(SeqIO.parse('bc25mer.240k.fasta','fasta'))
    big_list_25mers=[]
    
    for i in range(0,len(big_list_25mers_all)):
        big_list_25mers.append(str(big_list_25mers_all[i].seq))
        
    K=20
    big_list_20mers=[]
        
    for trial_25mer in big_list_25mers:
        trial_20mers = [trial_25mer[i: j] for i in range(len(trial_25mer)) for j in range(i + 1, len(trial_25mer) + 1) if len(trial_25mer[i:j]) == K]
        for trial_20mer in trial_20mers:
            if not ('G' in trial_20mer):
                #trial_20mer.replace('C','G')
                big_list_20mers.append(trial_20mer)
            
    pass_list=[]
    for probe in big_list_20mers:
        if not (('CCC' in probe) or ('TTT' in probe) or ('AAA' in probe)):
            pass_list.append(probe)
                
    pass_list2=[]
    for probe in pass_list:
        gc_count = gcCheck(probe)

        if (gc_count>=40) and (gc_count<=50):
            pass_list2.append(probe)
    
    pass_list3=[]
    i=0
    for probe in pass_list2:
        
        if (os.path.exists('readout_test_h3g8.fasta')):
            os.remove('readout_test_h3g8.fasta')
                
        if(os.path.exists('readout_check_hg38.xml')):
            os.remove('readout_check_hg38.xml')

        with open('readout_test_h3g8.fasta','w') as output_handle:
            record = SeqRecord(Seq(probe,generic_dna),id=probe+'_'+str(i))
            SeqIO.write(record,output_handle,'fasta')
        i+=1
        
        blastn_cline = blastn(query='readout_test_h3g8.fasta',db='/home/dps/merfish/blastdb/hg38/GRCh38',
                              out='readout_check_hg38.xml',dust='no',word_size=10,outfmt=5)
        stdout,stderr = blastn_cline()
        
        with open('readout_check_hg38.xml','r') as input_handle_hg38:
            blast_qresult = SearchIO.read(input_handle_hg38, 'blast-xml')

        flagged=True

        for hit in blast_qresult:
            if (hit.seq_len>=10):
                flagged=False
                break
                
        if flagged:
            pass_list3.append(probe)
            
    pass_list4=[]
    i=0
    for probe in pass_list3:

        if (os.path.exists('readout_test_ncRNA.fasta')):
            os.remove('readout_test_ncRNA.fasta')
                
        if(os.path.exists('readout_check_ncRNA.xml')):
            os.remove('readout_check_ncRNA.xml')
        
        with open('readout_test_ncRNA.fasta','w') as output_handle:
            record = SeqRecord(Seq(probe,generic_dna),id=probe+'_'+str(i))
            SeqIO.write(record,output_handle,'fasta')
        i+=1
                
        blastn_cline = blastn(query='readout_test_ncRNA.fasta',db='/home/dps/merfish/blastdb/tRNA/db/tRNA',
                              out='readout_check_ncRNA.xml',dust='no',word_size=10,outfmt=5)
        stdout,stderr = blastn_cline()
        
        with open('readout_check_ncRNA.xml','r') as input_handle_ncRNA:
            blast_qresult_ncRNA = SearchIO.read(input_handle_ncRNA, 'blast-xml')

        flagged=True

        for hit in blast_qresult_ncRNA:
            if (hit.seq_len>=10):
                flagged=False
                break
                
        if flagged:
            pass_list4.append(probe)
            
            
    if (os.path.exists('/home/dps/merfish/blastdb/readout/readout_candidates.fasta')):
        os.remove('/home/dps/merfish/blastdb/readout/readout_candidates.fasta')
            
    i=0
    with open('/home/dps/merfish/blastdb/readout/readout_candidates.fasta','a') as output_handle:
        for probe in pass_list4:
            record = SeqRecord(Seq(probe,generic_dna),id=probe+'_'+str(i),description='potential_readout+'+str(i))
            SeqIO.write(record,output_handle,"fasta")
            i+=1

    cline = NcbimakeblastdbCommandline(dbtype='nucl',input_file='/home/dps/merfish/blastdb/readout/readout_candidates.fasta',
                                       title='readout',out='/home/dps/merfish/blastdb/readout/db/readout')
    stdout, stderr = cline()
                
    pass_list5=[]
    i=0
    for probe in pass_list4:

        if (os.path.exists('readout_test_final.fasta')):
            os.remove('readout_test_final.fasta')
                
        if(os.path.exists('readout_test_final.xml')):
            os.remove('readout_test_final.xml')
        
        with open('readout_test_final.fasta','w') as output_handle:
            record = SeqRecord(Seq(probe,generic_dna),id=probe+'_'+str(i))
            SeqIO.write(record,output_handle,'fasta')
        i+=1
                
        blastn_cline = blastn(query='readout_test_final.fasta',db='/home/dps/merfish/blastdb/readout/db/readout',
                              out='readout_test_final.xml',dust='no',word_size=10,outfmt=5)
        stdout,stderr = blastn_cline()
        
        with open('readout_test_final.xml','r') as input_handle_final:
            blast_qresult_final = SearchIO.read(input_handle_final, 'blast-xml')
        
        flagged=True

        for hit in blast_qresult_final:
            if (hit.seq_len>=10) and (hit.seq_len<20):
                flagged=False
                break
                
        if flagged:
            pass_list5.append(probe)
    
    # place each probe in dataframe 
    df_readout_probes = pd.DataFrame(columns=['probe','sequence'])
    i=0
    for probe in pass_list5:
        df_readout_probes = df_readout_probes.append({'probe': i, 'sequence': probe},ignore_index=True)
        i+=1
        
    return df_readout_probes

## Published standard readout sequences
1. Load readout sequences used for 16-bit MHD4 code from [Moffitt et al 2018](https://doi.org/10.1073/pnas.1617699113).
2. Need to use reverse complement of these when assembling probes.
  
Moffitt et al used 'A','T','G' instead of the 'A','T','C' strategy used for our denovo readout sequence design.

In [8]:
def load_moffitt_readout_sequences():
    
    df_readout = pd.read_csv('/home/dps/merfish/16bit_MHD4_readout.tsv',sep='\t',header=0,index_col='probe')
    df_readout.reset_index(inplace=True)
    
    return df_readout

## Published amplified readout sequences 
1. Load 5x5 amplified readout sequences for 16-bit MHD4 code from [Xia et al 2019](https://doi.org/10.1038/s41598-019-43943-8).
2. Need to use reverse complement of original readout sequences when assembling probes.

This is a clear area of opportunity. Look into orthogonal strategies used in [SABER](https://www.nature.com/articles/s41592-019-0404-0#Sec36). I agree with Moffitt that solid-phase amplifiers with defined number of binding sites makes more sense as it will reduce variation. Even if it does cost a little bit extra. Also, many groups I have talked to have not been able to successfully perform the SABER reaction to get amplifiers.  
  
For single shot multiplexing with amplification, we often use [HCR v3.0](https://www.molecularinstruments.com/hcr-v3). Since HCR v3.0 is also an unbounded amplification, it has similiar issues for quantification and usability for multiplex by barcoding. Given the known issues with seqFISH and seqFISH+, I'm hesistant to use HCR based strategies for this purpose.

In [9]:
def load_amplified_readout_sequences():
    
    df_readout_amplified = pd.read_excel('/home/dps/merfish/16bit_MHD4_amplify.xlsx',skip_row=0,header=1,index_col='Bit')
    
    return df_readout_amplified

## Wrapper function to load selected readout probes

In [10]:
def load_readout_probes(readout_strategy):
    
    if readout_strategy=='denovo':
        df_readout_probes=generate_denovo_readout_probes()
    elif readout_strategy=='standard':
        df_readout_probes=load_moffitt_readout_sequences()
    else:
        df_readout_probes=load_amplified_readout_sequences()
        
    return df_readout_probes

## Wrapper function to load selected Hamming codes
Load:
1. 14-bit modified hamming codes from [Zhuang lab example data #1](http://zhuang.harvard.edu/merfish.html).
2. 16-bit modified hamming codes from [Zhuang lab example data #1](http://zhuang.harvard.edu/merfish.html).
3. 18-bit modified hamming codes from [Wollman MERFISH github repo](https://github.com/wollmanlab/PySpots).
4. 24-bit modified hamming codes from [Wollman MERFISH github repo](https://github.com/wollmanlab/PySpots).  
  
This is a clear area of opportunity. We are working on alternative coding/decoding at ASU, but are not ready to roll it out yet.

In [11]:
def load_hamming_codes(bit):

    if (bit=='14-bit'):
        list_hamming=[]
        with open('/home/dps/merfish/codebook/14bit_MHD2_codebook.fasta','r') as input_handle:
            for record in SeqIO.parse(input_handle, 'fasta'):
                list_hamming.append(record.description.split())

        df_hamming_codes=pd.DataFrame(list_hamming,columns=['R0','R1','R2','R3',
                                                        'R4','R5','R6','R7',
                                                        'R8','R9','R10','R11',
                                                        'R12','R13'])
    elif (bit=='16-bit'):
        list_hamming=[]
        with open('/home/dps/merfish/codebook/16bit_MHD4_codebook.fasta','r') as input_handle:
            for record in SeqIO.parse(input_handle, 'fasta'):
                list_hamming.append(record.description.split())

        df_hamming_codes=pd.DataFrame(list_hamming,columns=['R0','R1','R2','R3',
                                                        'R4','R5','R6','R7',
                                                        'R8','R9','R10','R11',
                                                        'R12','R13','R14','R15'])
        
    elif (bit=='18-bit'):
        df_hamming_codes=pd.DataFrame(columns=['R0','R1','R2','R3',
                                               'R4','R5','R6','R7',
                                               'R8','R9','R10','R11',
                                               'R12','R13','R14','R15',
                                               'R16','R17'])
        with open('/home/dps/merfish/codebook/18bit_MHD4_codebook.fasta','r') as input_handle:
            df_hamming_codes=pd.read_csv(input_handle,delimeter=',')
                
    else:
        df_hamming_codes=pd.DataFrame(columns=['R0','R1','R2','R3',
                                               'R4','R5','R6','R7',
                                               'R8','R9','R10','R11',
                                               'R12','R13','R14','R15',
                                               'R16','R17','R18','R19',
                                               'R20','R21','R22','R23'])
        with open('/home/dps/merfish/codebook/24bit_MHD4_codebook.fasta','r') as input_handle:
            df_hamming_codes=pd.read_csv(input_handle,delimeter=',')
            
    return df_hamming_codes

## Encoding + readout assembly using sequential labeling
1. Create experiment specific codebook using four-colors, number of genes, and relative expression levels
2. Sort genes into bins of 4 with different expression levels
3. Assign highest expression to lowest efficiency channel on OPM (Cy7), second highest to channel with most autofluorescence (Alexa 488), third highest to Alexa 647, and lowest to Alexa 594.
4. Save metadata and encoding+readout probes into Pandas dataframe

In [12]:
def generate_readout_plus_encoding_sequential(df_readout,df_encoding,relative_expression,readout_type,number_of_colors):
    
    # determine number of rounds
    gene_ids = df_encoding['gene'].unique()
    number_of_genes=len(gene_ids)
    number_of_rounds = number_of_genes/number_of_colors
    
    # sort genes based on relative expression (high -> low)
    zipped_genes_expression = zip(relative_expression, gene_ids)
    sorted_genes_expression = sorted(zipped_genes_expression)
    sorted_genes = [element for _, element in sorted_genes_expression]
    
    # split genes into rounds 
    split_gene_ids=np.array_split(sorted_genes, number_of_colors)

    # if using amplified readout, clean up to just have primary readouts
    if readout_type == 'amplified':
        cols=df_readout.columns
        df_readout = df_readout.drop(columns=[str(cols[1]),str(cols[2]), str(cols[3])])
        df_readout = df_readout.rename(columns={str(cols[0]): 'sequence'})
        df_readout.reset_index(inplace=True)
        
    # create metadata dataframe
    df_metadata = pd.DataFrame(columns=['gene','readout round','readout bit','readout dye'])

    # create encoding+readout dataframe
    df_encoding_readout = pd.DataFrame(columns=['probe','gene','sequence','isoform','readout round', 'readout bit', 'readout dye'])
    
    
    
    # assign highest expression across number of rounds, then repeat with next expression
    # until rounds are full or genes are exhausted
    
    # in this strategy, we assign both readout flaps to the same readout sequence
    readout_dyes=['alexa750','alexa488','alexa647','alexa594']
    bit_counter = 0
    
    for gene_ids, readout_dye in zip(split_gene_ids,readout_dyes):
        round_counter=0

        for gene_id in gene_ids:
            # extract encoding probes for this gene
            df_gene = df_encoding[df_encoding['gene']==gene_id]
            df_gene.reset_index(inplace=True)
            
            df_temp=pd.DataFrame(columns=['probe','gene','sequence','isoform','readout bit','readout dye'])
        
            number_of_probes = len(df_gene['sequence'].unique())
            
            for i in range(0,number_of_probes):
                readout = Seq(df_readout['sequence'][np.int(bit_counter)], generic_dna)
                df_temp=df_temp.append({'probe': i,
                                        'gene': df_gene['gene'][i],
                                        'sequence': str(readout.reverse_complement())
                                        + 'A'+ df_gene['sequence'][i] + 'A'
                                        + str(readout.reverse_complement()),
                                        'isoform': df_gene['isoform'][i],
                                        'readout round': round_counter,
                                        'readout bit': bit_counter,
                                        'readout dye': readout_dye},ignore_index=True)
                # create metadata structure
                df_metadata=df_metadata.append({'gene': gene_id, 'readout round': round_counter,
                                                'readout bit': bit_counter, 
                                                'readout dye': readout_dye},ignore_index=True)
            round_counter+=1
            bit_counter+=1
            df_encoding_readout = df_encoding_readout.append(df_temp)
        
            
    df_encoding_readout.reset_index(inplace=True)
    df_encoding_readout=df_encoding_readout.drop(columns=['index'])
    
    return df_encoding_readout, df_metadata

## Encoding + readout assembly using Hamming codes (aka codebook)
1. Create experiment specific codebook using two-colors, number of genes, and selected Hamming codes.
2. Assemble encoding and readout probes
3. Save metadata and encoding+readout probes into Pandas dataframe

In [13]:
def generate_readout_plus_encoding_barcoded(df_readout,df_encoding, df_hamming_codes, barcode_type, readout_type):
    
    # randomly assign genes to barcode sequences
    number_of_barcode_seqs = len(df_hamming_codes)
    gene_ids = df_encoding['gene'].unique()
    number_of_genes=len(gene_ids)
    barcode_seq_assignments=np.random.choice(number_of_barcode_seqs,number_of_genes,replace=False)
    
    if readout_type == 'amplified':
        cols=df_readout.columns
        df_readout = df_readout.drop(columns=[str(cols[1]),str(cols[2]), str(cols[3])])
        df_readout = df_readout.rename(columns={str(cols[0]): 'sequence'})
        df_readout.reset_index(inplace=True)

    #create metadata
    df_metadata = pd.DataFrame(columns=['gene','barcode ID','hamming type','readout type'])

    df_encoding_readout = pd.DataFrame(columns=['probe','gene','sequence','isoform','readout 1','readout 2'])

    # loop through all genes
    for gene_id, assignment in zip(gene_ids,barcode_seq_assignments):

        # load barcode sequence associated with this gene
        bits = df_hamming_codes.iloc[assignment]

        # extract encoding probes for this gene
        df_gene = df_encoding[df_encoding['gene']==gene_id]
        df_gene.reset_index(inplace=True)

        # extract number of encoding probes and calculate halfway point
        number_of_probes = len(df_gene['sequence'].unique())
        halfway = number_of_probes//2

        df_temp=pd.DataFrame(columns=['probe','gene','sequence','isoform','readout 1','readout 2'])

        # loop over sequence
        # place the first two bits on first half the probes (3' then 5')
        # place the second two bits on second half the probes (3' then 5')
        bit_placed=0
        bit_counter=0
        
        for bit in bits:
            if (np.int(bit)==1):
                if bit_placed == 0:
                    for i in range(0,halfway):
                        readout = Seq(df_readout['sequence'][np.int(bit_counter)], generic_dna)
                        df_temp=df_temp.append({'probe': i,
                                                'gene': df_gene['gene'][i], 
                                                'sequence': str(readout.reverse_complement())
                                                    + 'A'+ df_gene['sequence'][i],
                                                'isoform': df_gene['isoform'][i],
                                                'readout 1': bit_counter,
                                                'readout 2': ''},ignore_index=True)
                    bit_placed=bit_placed+1
                elif bit_placed == 1:
                    for i in range(0,halfway):
                        readout = Seq(df_readout['sequence'][np.int(bit_counter)], generic_dna)
                        df_temp['sequence'][i]= df_temp['sequence'][i]+'A'+ str(readout.reverse_complement())
                        df_temp['readout 2'][i]= bit_counter
                    bit_placed=bit_placed+1
                elif bit_placed == 2:
                    for i in range(halfway,len(df_gene)):
                        readout = Seq(df_readout['sequence'][np.int(bit_counter)], generic_dna)
                        df_temp=df_temp.append({'probe': i,
                                                'gene': df_gene['gene'][i], 
                                                'sequence': str(readout.reverse_complement())
                                                    + 'A' + df_gene['sequence'][i],
                                                 'isoform': df_gene['isoform'][i],
                                                'readout 1': bit_counter,
                                                'readout 2': ''}, ignore_index=True)
                    bit_placed=bit_placed+1
                else:
                    for i in range(halfway,len(df_gene)):
                        readout = Seq(df_readout['sequence'][np.int(bit_counter)], generic_dna)
                        df_temp['sequence'][i]= df_temp['sequence'][i]+'A'+ str(readout.reverse_complement())
                        df_temp['readout 2'][i]= bit_counter
            bit_counter=bit_counter+1

        df_encoding_readout = df_encoding_readout.append(df_temp)

        # create metadata structure
        df_metadata=df_metadata.append({'gene': gene_id, 'barcode ID': assignment,
                                        'hamming type': barcode_type, 'readout type': readout_type},ignore_index=True)

    df_encoding_readout.reset_index(inplace=True)
    df_encoding_readout=df_encoding_readout.drop(columns=['index'])
    
    return df_encoding_readout, df_metadata

## Denovo primer design
1. Read (or create) set of 20-mers from known set of 25-mers
2. Select 20-mers that end in 'GX' or 'CX' or 'XG' or 'XC' (GC clamp)
3. Select 20-mers that do not contain triplet nucleotides
4. Select 20-mers that have 50-65% GC
5. Select 20-mers with Tm between 70 and 72C at 390 mM salt
6. BLAST 20-mers against lincRNA, rRNA, tRNA, mt-tRNA, mt-RNA for species of interest. Select all probes with less than 8 hits.
7. BLAST 20-mers against other 20-mers. Select all probes with less than 8 hits.
8. BLAST 20-mers against encoding+readout probes. Select all probes with less than 8 hits.
9. Select forward and reverse primer based on minimum number of off-target hits.
10. Save primers into Pandas dataframe

In [14]:
def generate_primers(df_probes):
    
    # Make sure non-coding RNA database exists
    fasta_to_blastDB_ncRNA()

    if not(os.path.exists('/home/dps/merfish/verified_primers.csv')):
        big_list_25mers_all = list(SeqIO.parse('bc25mer.240k.fasta','fasta'))
        big_list_25mers=[]

        for i in range(0,len(big_list_25mers_all)):
            big_list_25mers.append(str(big_list_25mers_all[i].seq))

        K=20
        big_list_20mers=[]

        # Truncate 25-mers into 20-mers that have GC clamp
        for trial_25mer in big_list_25mers:
            trial_20mers = [trial_25mer[i: j] for i in range(len(trial_25mer)) for j in range(i + 1, len(trial_25mer) + 1) if len(trial_25mer[i:j]) == K]
            for trial_20mer in trial_20mers:
                if (trial_20mer.endswith('G') or trial_20mer.endswith('C') 
                    or str(trial_20mer[:-1]).endswith('G') or str(trial_20mers[:-1]).endswith('C')):
                        big_list_20mers.append(trial_20mer)

        # Check for triplets
        pass_list=[]
        for primer in big_list_20mers:
            if not (('CCC' in primer) or ('TTT' in primer) or ('AAA' in primer) or ('GGG' in primer)):
                pass_list.append(primer)

        # Check GC content
        pass_list2=[]
        for primer in pass_list:
            gc_count = gcCheck(primer)
            if (gc_count>=50) and (gc_count<=65):
                pass_list2.append(primer)

        # Check melting temperature
        pass_list3=[]
        for primer in pass_list2:
            tmval = mt.Tm_NN(primer,Na=390,dnac1=25,dnac2=0)        
            if (tmval>=70.0) and (tmval<72.0):
                    pass_list3.append(primer)

        # BLAST against transcriptome
        pass_list4=[]
        i=0
        for primer in pass_list3:

            if (os.path.exists('primer_test_h3g8.fasta')):
                os.remove('primer_test_h3g8.fasta')
                
            if(os.path.exists('primer_check_hg38.xml')):
                os.remove('primer_check_hg38.xml')

            with open('primer_test_h3g8.fasta','w') as output_handle:
                record = SeqRecord(Seq(primer,generic_dna),id='primer_'+str(i))
                SeqIO.write(record,output_handle,'fasta')
            i+=1

            blastn_cline = blastn(query='primer_test_h3g8.fasta',db='/home/dps/merfish/blastdb/hg38/GRCh38',
                                  out='primer_check_hg38.xml',dust='no',word_size=10,outfmt=5)
            stdout,stderr = blastn_cline()

            with open('primer_check_hg38.xml','r') as input_handle_hg38:
                blast_qresult = SearchIO.read(input_handle_hg38, 'blast-xml')

            flagged=True

            for hit in blast_qresult:
                if (hit.seq_len>=8) and (hit.seq_len<20):
                    flagged=False
                    break

            if flagged:
                pass_list4.append(primer)

        # BLAST against non-coding RNA database
        pass_list5=[]
        i=0
        for primer in pass_list4:

            if (os.path.exists('primer_test_ncRNA.fasta')):
                os.remove("primer_test_ncRNA.fasta")
                
            if(os.path.exists('primer_check_ncRNA.xml')):
                os.remove('primer_check_ncRNA.xml')
            
            with open('primer_test_ncRNA.fasta','w') as output_handle:
                record = SeqRecord(Seq(primer,generic_dna),id='primer_'+str(i))
                SeqIO.write(record,output_handle,'fasta')
            i+=1

            blastn_cline = blastn(query='primer_test_ncRNA.fasta',db='/home/dps/merfish/blastdb/tRNA/db/tRNA',
                                  out='primer_check_ncRNA.xml',dust='no',word_size=10,outfmt=5)
            stdout,stderr = blastn_cline()

            with open('primer_check_ncRNA.xml','r') as input_handle_ncRNA:
                blast_qresult_ncRNA = SearchIO.read(input_handle_ncRNA, 'blast-xml')

            flagged=True

            for hit in blast_qresult_ncRNA:
                if (hit.seq_len>=8) and (hit.seq_len<20):
                    flagged=False
                    break

            if flagged:
                pass_list5.append(primer)

        # create BLASTDB for remaining primers
        if (os.path.exists('/home/dps/merfish/blastdb/primer/primer_candidates.fasta')):
                os.remove("/home/dps/merfish/blastdb/primer/primer_candidates.fasta")
        
        i=0
        with open('/home/dps/merfish/blastdb/primer/primer_candidates.fasta','a') as output_handle:   
            for primer in pass_list5:
                record = SeqRecord(Seq(primer,generic_dna),id=primer+'_'+str(i),description="potential_primer_"+str(i))
                SeqIO.write(record,output_handle,"fasta")
                i+=1

        cline = NcbimakeblastdbCommandline(dbtype='nucl',input_file='/home/dps/merfish/blastdb/primer/primer_candidates.fasta',
                                           title='primer',out='/home/dps/merfish/blastdb/primer/db/primer')
        stdout, stderr = cline()

        # BLAST against remaining primer list
        pass_list6=[]
        i=0
        for primer in pass_list5:
            if (os.path.exists('primer_test_final.fasta')):
                os.remove("primer_test_final.fasta")
                
            if(os.path.exists('primer_test_final.xml')):
                os.remove('primer_test_final.xml')
            
            with open('primer_test_final.fasta','w') as output_handle:
                record = SeqRecord(Seq(primer,generic_dna),id='primer_'+str(i))
                SeqIO.write(record,output_handle,'fasta')
            i+=1

            blastn_cline = blastn(query='primer_test_final.fasta',db='/home/dps/merfish/blastdb/primer/db/primer',
                                  out='primer_test_final.xml',dust='no',word_size=10,outfmt=5)
            stdout,stderr = blastn_cline()

            with open('primer_test_final.xml','r') as input_handle_final:
                blast_qresult_final = SearchIO.read(input_handle_final, 'blast-xml')

            flagged=True

            for hit in blast_qresult_final:
                if (hit.seq_len>=8) and (hit.seq_len<20):
                    flagged=False
                    break

            if flagged:
                pass_list6.append(primer)
                
        # create dataframe
        df_checked_primers = pd.DataFrame(columns=['primer','sequence'])
        i=0
        for primers in pass_list6:
            df_checked_primers = df_checked_primers.append({'primer': i, 'sequence': primers},ignore_index=True)
            i+=1
        with open('/home/dps/merfish/verified_primers.csv','w') as output_handle:
            df_checked_primers.to_csv(output_handle,index=False)
        
    else:
        with open('/home/dps/merfish/verified_primers.csv','r') as input_handle:
            df_checked_primers = pd.read_csv(input_handle)
            
        pass_list6 = df_checked_primers['sequence']
        
    # Create BLASTDB for encoding+readout probe constructs    
    if (os.path.exists('probe_list.fasta')):
        os.remove("probe_list.fasta")
        
    extracted_probes = df_probes["sequence"]
    
    i=0
    with open('probe_list.fasta','w') as output_handle:   
        for probe in extracted_probes:
            record = SeqRecord(Seq(probe,generic_dna),id=probe+'_'+str(i),description="probe_"+str(i)) 
            SeqIO.write(record,output_handle,"fasta")
            i+=1

    cline = NcbimakeblastdbCommandline(dbtype='nucl',input_file='probe_list.fasta',
                                       title='probes',out='/home/dps/merfish/blastdb/probes/db/probes')
    stdout, stderr = cline()
    
    # BLAST against encoding+readout probe constructs
    pass_list7=[]
    i=0
    for primer in pass_list6:

        if (os.path.exists('primer_test_probes.fasta')):
            os.remove("primer_test_probes.fasta")
            
        if(os.path.exists('primer_test_probes.xml')):
                os.remove('primer_test_probes.xml')
        
        with open('primer_test_probes.fasta','w') as output_handle:
            record = SeqRecord(Seq(primer,generic_dna),id='primer_'+str(i))
            SeqIO.write(record,output_handle,'fasta')
        i+=1
                
        blastn_cline = blastn(query='primer_test_probes.fasta',db='/home/dps/merfish/blastdb/probes/db/probes',
                              out='primer_test_probes.xml',dust='no',word_size=10,outfmt=5)
        stdout,stderr = blastn_cline()
        
        with open('primer_test_probes.xml','r') as input_handle_final:
            blast_qresult_final = SearchIO.read(input_handle_final, 'blast-xml')
        
        flagged=True

        for hit in blast_qresult_final:
            if (hit.seq_len>=8) and (hit.seq_len<20):
                flagged=False
                break
                
        if flagged:
            pass_list7.append(primer)
            
    # place final primers into a dataframe
    df_final_primers = pd.DataFrame(columns=['primer','sequence'])
    i=0
    for primers in pass_list7:
        df_final_primers = df_final_primers.append({'primer': i, 'sequence': primers},ignore_index=True)
        i+=1
        
    return df_final_primers

## Add primers, T7 promoter, and linkers to encoding + readout probe sequences for ordering
- 5' - forward primer -'a'- readout 1 -'a'- encoding -'a'- readout 2 -'a'- T7 -'a'- reverse primer - 3'
- when constructing, make sure that no triplet sequences are introduced

In [15]:
def construct_full_probes(df_encodingreadout,df_primers):
    
    # extract two random primers
    primer_assignments=np.random.choice(len(df_primers),2,replace=False)
    primer_01 = primer_assignments[0]
    primer_02 = primer_assignments[1]
    
    # T7 promoter sequence
    T7_sequence = 'TAATACGACTCACTATAGGG'
    
    df_fullprobes = pd.DataFrame(columns=['probe','gene','sequence','isoform'])
    
    for i in range(0,len(df_encodingreadout)):
        df_fullprobes=df_fullprobes.append({'probe': i,
                                            'gene': df_encodingreadout['gene'][i],
                                            'sequence': df_primers['sequence'][primer_01] + 'A'
                                            + df_encodingreadout['sequence'][i] + 'A'
                                            + T7_sequence + 'A'
                                            + df_primers['sequence'][primer_02],
                                            'isoform': df_encodingreadout['isoform'][i]},
                                            ignore_index=True)
    
    df_primer_choice = pd.DataFrame(columns=['index','sequence'])
    df_primer_choice = df_primer_choice.append({'index': 0, 
                                                'sequence': df_primers['sequence'][primer_01]},
                                               ignore_index=True)
    df_primer_choice = df_primer_choice.append({'index': 1, 
                                                'sequence': df_primers['sequence'][primer_02]},
                                               ignore_index=True)
    
    
    return df_fullprobes, df_primer_choice

## Run probe design pipeline
1. Generate encoding probes for specified genes
2. Load requested readout probes
3. Generate encoding+readout probes for requested strategy
4. Generate primers (can be slow if pre-screened database does not exist already)
5. Generate full probes for ordering

In [16]:
# generate encoding probe sequences
df_encoding_probes = generate_encoding_probes(genome,gene_ids)

# load (or generate) readout probe sequences
df_readout_probes = load_readout_probes(readout_strategy[1])

# generate multiplex by sequential readout + encoding
if readout_strategy[0]=='sequential':
    df_encoding_readout_probes, df_metadata = generate_readout_plus_encoding_sequential(df_readout_probes,
                                                                                        df_encoding_probes,
                                                                                        relative_expression,
                                                                                        readout_strategy[1],
                                                                                        number_of_colors)
# generate multiplex by barcoding readout + encoding
else:
    df_barcodes = loadHammingFromDisk(hamming_bit)
    # generate encoding+readout probe sequences using codebook
    df_encoding_readout_probes, df_metadata = generateReadoutPlusEncoding(df_readout_probes,
                                                                          df_encoding_probes,
                                                                          df_barcodes,
                                                                          hamming_bit,
                                                                          readout_strategy[1])

# generate forward and reverse primer sequences for this set of encoding+readout probes
df_primers = generate_primers(df_encoding_readout_probes)

# generate full probe sequences for ordering
df_full_probes, df_primer_choice = construct_full_probes(df_encoding_readout_probes,df_primers)

[array(['CLDN5'], dtype='<U6'), array(['PTPRC'], dtype='<U6'), array(['PDGFRB'], dtype='<U6'), array(['CDH1'], dtype='<U6')]


/home/dps/miniconda3/envs/merfish/lib/python3.6/site-packages/Bio/SearchIO/BlastIO/blast_xml.py:364: BiopythonParserWarning: Renaming hit ID '' to a BLAST-generated ID 'gnl|BL_ORD_ID|9' since the ID was already matched by your query 'primer_1718'. Your BLAST database may contain duplicate entries.
  BiopythonParserWarning,


## Save all dataframes to disk with unique identifier and all settings needed to regenerate workflow

In [18]:
# pull date
# TO DO: look into better identifier method
d = datetime.now()
prefix = d.isoformat()

# write encoding probes
encoding_file = prefix+'encoding_probes.csv'
with open(encoding_file,'w') as output_handle:
    df_encoding_probes.to_csv(output_handle,index=False)
    
# write readout probes
readout_file = prefix+'readout_file.csv'
with open(readout_file,'w') as output_handle:
    df_readout_probes.to_csv(output_handle,index=False)

# write readout probes
encoding_readout_file = prefix+'encoding_readout_file.csv'
with open(encoding_readout_file,'w') as output_handle:
    df_encoding_readout_probes.to_csv(output_handle,index=False)
    
# write hamming codes
if readout_strategy[0]=='barcode':
    hamming_file = prefix+'hamming_file.csv'
    with open(hamming_file,'w') as output_handle:
        df_barcodes.to_csv(output_handle,index=False)
    
# write primers
primer_file = prefix+'primer_file.csv'
with open(primer_file,'w') as output_handle:
    df_primer_choice.to_csv(output_handle,index=False)
    
# write full probes (T7, primers, encoding, readout)
fullprobes_file = prefix+'fullprobes_file.csv'
with open(fullprobes_file,'w') as output_handle:
    df_full_probes.to_csv(output_handle,index=False)
    
# write metadata
metadata_file = prefix+'metadata_file.csv'
with open(metadata_file,'w') as output_handle:
    df_metadata.to_csv(output_handle,index=False)

## License

Copyright (c) 2020 Douglas Shepherd

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.